In [1]:
from pyspark.sql import SparkSession
import os
from pyspark.sql.types import StructType, StructField, IntegerType, StringType, DateType
from pyspark.sql.functions import from_json, col, window, count, explode, expr
import tempfile
from ast import literal_eval

In [2]:
import os
import sys

os.environ['PYSPARK_PYTHON'] = sys.executable
os.environ['PYSPARK_DRIVER_PYTHON'] = sys.executable

In [3]:
# Create the Spark Session
from pyspark.sql import SparkSession

spark = SparkSession \
    .builder \
    .appName("Streaming from Kafka") \
    .config("spark.streaming.stopGracefullyOnShutdown", True) \
    .config('spark.jars.packages', 'org.apache.spark:spark-sql-kafka-0-10_2.12:3.5.1') \
    .config('spark.hadoop.fs.gs.impl', 'com.google.cloud.hadoop.fs.gcs.GoogleHadoopFileSystem') \
    .config('spark.jars', 'https://storage.googleapis.com/hadoop-lib/gcs/gcs-connector-hadoop3-latest.jar') \
    .config("spark.sql.shuffle.partitions", 4) \
    .master("local[*]") \
    .getOrCreate()

spark

24/07/28 15:50:56 WARN Utils: Your hostname, rihab-X556UJ resolves to a loopback address: 127.0.1.1; using 192.168.1.10 instead (on interface wlp3s0)
24/07/28 15:50:57 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
:: loading settings :: url = jar:file:/home/rihab/spark-3.3.2-bin-hadoop3/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/rihab/.ivy2/cache
The jars for the packages stored in: /home/rihab/.ivy2/jars
org.apache.spark#spark-sql-kafka-0-10_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-798482e1-26ad-4643-9257-130c12721a9e;1.0
	confs: [default]
	found org.apache.spark#spark-sql-kafka-0-10_2.12;3.5.1 in central
	found org.apache.spark#spark-token-provider-kafka-0-10_2.12;3.5.1 in central
	found org.apache.kafka#kafka-clients;3.4.1 in central
	found org.lz4#lz4-java;1.8.0 in central
	found org.xerial.snappy#snappy-java;1.1.10.3 in central
	found org.slf4j#slf4j-api;2.0.7 in central
	found org.apache.hadoop#hadoop-client-runtime;3.3.4 in central
	found org.apache.hadoop#hadoop-client-api;3.3.4 in central
	found commons-logging#commons-logging;1.1.3 in central
	found com.google.code.findbugs#jsr305;3.0.0 in central
	found org.apache.commons#commons-pool2;2.11.1 in central
:: resolution report :: resolve 719ms :: artifacts dl 24ms
	:: m

24/07/28 15:50:58 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


24/07/28 15:52:04 ERROR Inbox: Ignoring error
java.lang.NullPointerException
	at org.apache.spark.storage.BlockManagerMasterEndpoint.org$apache$spark$storage$BlockManagerMasterEndpoint$$register(BlockManagerMasterEndpoint.scala:582)
	at org.apache.spark.storage.BlockManagerMasterEndpoint$$anonfun$receiveAndReply$1.applyOrElse(BlockManagerMasterEndpoint.scala:123)
	at org.apache.spark.rpc.netty.Inbox.$anonfun$process$1(Inbox.scala:103)
	at org.apache.spark.rpc.netty.Inbox.safelyCall(Inbox.scala:213)
	at org.apache.spark.rpc.netty.Inbox.process(Inbox.scala:100)
	at org.apache.spark.rpc.netty.MessageLoop.org$apache$spark$rpc$netty$MessageLoop$$receiveLoop(MessageLoop.scala:75)
	at org.apache.spark.rpc.netty.MessageLoop$$anon$1.run(MessageLoop.scala:41)
	at java.base/java.util.concurrent.Executors$RunnableAdapter.call(Executors.java:515)
	at java.base/java.util.concurrent.FutureTask.run(FutureTask.java:264)
	at java.base/java.util.concurrent.ThreadPoolExecutor.runWorker(ThreadPoolExecutor.

In [9]:
spark._jsc.hadoopConfiguration().set("google.cloud.auth.service.account.json.keyfile", "/home/rihab/Downloads/golden-system-411709-5116d50b308f.json") 

In [5]:
# Create the streaming_df to read from kafka
streaming_df = spark.readStream\
    .format("kafka") \
    .option("kafka.bootstrap.servers", "localhost:9092") \
    .option("subscribe", "velib_bikes") \
    .option("startingOffsets", "earliest") \
    .load()

In [5]:
df = streaming_df.selectExpr("CAST(value AS STRING)")

In [11]:
bikes_query = df.writeStream.queryName("bikes10").format("memory").start()

24/07/27 21:40:36 WARN ResolveWriteToStream: Temporary checkpoint location created which is deleted normally when the query didn't fail: /tmp/temporary-ba7828fa-a180-45ed-a6e8-6eb5a9d05225. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folder is best effort.
24/07/27 21:40:36 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.


In [13]:
bikes= spark.sql("select * from bikes10")
bikes.show()

+--------------------+
|               value|
+--------------------+
|{"total_count": 1...|
|{"total_count": 1...|
+--------------------+



In [18]:
rdds = bikes.rdd.map(lambda bike: bike['value'])

In [19]:
rdds

PythonRDD[15] at RDD at PythonRDD.scala:53

In [20]:
data = rdds.collect()

In [33]:
data

['{"total_count": 1496, "results": [{"stationcode": "44015", "name": "Rouget de L\'isle - Watteau", "is_installed": "OUI", "capacity": 20, "numdocksavailable": 4, "numbikesavailable": 15, "mechanical": 11, "ebike": 4, "is_renting": "OUI", "is_returning": "OUI", "duedate": "2024-07-27T15:04:02+00:00", "coordonnees_geo": {"lon": 2.3963020229163, "lat": 48.778192750803}, "nom_arrondissement_communes": "Vitry-sur-Seine", "code_insee_commune": null}]}',
 '{"total_count": 1496, "results": [{"stationcode": "44015", "name": "Rouget de L\'isle - Watteau", "is_installed": "OUI", "capacity": 20, "numdocksavailable": 4, "numbikesavailable": 16, "mechanical": 11, "ebike": 5, "is_renting": "OUI", "is_returning": "OUI", "duedate": "2024-07-27T19:05:12+00:00", "coordonnees_geo": {"lon": 2.3963020229163, "lat": 48.778192750803}, "nom_arrondissement_communes": "Vitry-sur-Seine", "code_insee_commune": null}]}']

In [37]:
rdds.toDF(schema=schema).show()

24/07/27 22:09:48 ERROR Executor: Exception in task 0.0 in stage 9.0 (TID 14)
org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/home/rihab/spark-3.3.2-bin-hadoop3/python/lib/pyspark.zip/pyspark/worker.py", line 686, in main
    process()
  File "/home/rihab/spark-3.3.2-bin-hadoop3/python/lib/pyspark.zip/pyspark/worker.py", line 678, in process
    serializer.dump_stream(out_iter, outfile)
  File "/home/rihab/spark-3.3.2-bin-hadoop3/python/lib/pyspark.zip/pyspark/serializers.py", line 273, in dump_stream
    vs = list(itertools.islice(iterator, batch))
  File "/home/rihab/spark-3.3.2-bin-hadoop3/python/lib/pyspark.zip/pyspark/util.py", line 81, in wrapper
    return f(*args, **kwargs)
  File "/home/rihab/.local/lib/python3.10/site-packages/pyspark/sql/session.py", line 910, in prepare
    verify_func(obj)
  File "/home/rihab/.local/lib/python3.10/site-packages/pyspark/sql/types.py", line 1722, in verify
    verify_value(obj)
  File "/home/rihab/.lo

Py4JJavaError: An error occurred while calling o234.showString.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 9.0 failed 1 times, most recent failure: Lost task 0.0 in stage 9.0 (TID 14) (192.168.1.10 executor driver): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/home/rihab/spark-3.3.2-bin-hadoop3/python/lib/pyspark.zip/pyspark/worker.py", line 686, in main
    process()
  File "/home/rihab/spark-3.3.2-bin-hadoop3/python/lib/pyspark.zip/pyspark/worker.py", line 678, in process
    serializer.dump_stream(out_iter, outfile)
  File "/home/rihab/spark-3.3.2-bin-hadoop3/python/lib/pyspark.zip/pyspark/serializers.py", line 273, in dump_stream
    vs = list(itertools.islice(iterator, batch))
  File "/home/rihab/spark-3.3.2-bin-hadoop3/python/lib/pyspark.zip/pyspark/util.py", line 81, in wrapper
    return f(*args, **kwargs)
  File "/home/rihab/.local/lib/python3.10/site-packages/pyspark/sql/session.py", line 91

In [26]:
import pandas as pd

In [41]:
pd.DataFrame(dict(bikes.toPandas()['value']), index=[0])

,0,1
0,"{""total_count"": 1496, ""results"": [{""stationcod...","{""total_count"": 1496, ""results"": [{""stationcod..."


In [28]:
sparkdf = spark.createDataFrame([list(bikes.toPandas()['value'])])

In [30]:
sparkdf.show()

+--------------------+--------------------+
|                  _1|                  _2|
+--------------------+--------------------+
|{"total_count": 1...|{"total_count": 1...|
+--------------------+--------------------+



In [64]:
schema1= StructType([StructField('total_count', StringType(), True),
                    StructField('results', StringType(), True)])

In [65]:
streaming_df.select(col('value').cast('string').alias('test')).writeStream.format("console").start()

24/07/27 19:28:42 WARN ResolveWriteToStream: Temporary checkpoint location created which is deleted normally when the query didn't fail: /tmp/temporary-20a999f9-f8d5-4505-9fea-7089215b7ff4. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folder is best effort.
24/07/27 19:28:42 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.


-------------------------------------------
Batch: 0
-------------------------------------------
+--------------------+
|                test|
+--------------------+
|{"total_count": 1...|
+--------------------+



In [66]:
streaming_df.select(col('value').cast('string')).writeStream.format("console").start()

24/07/27 19:28:45 WARN ResolveWriteToStream: Temporary checkpoint location created which is deleted normally when the query didn't fail: /tmp/temporary-ffafa7df-d2c9-47b9-809d-ade13275acf4. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folder is best effort.
24/07/27 19:28:45 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.


-------------------------------------------
Batch: 0
-------------------------------------------
+--------------------+
|               value|
+--------------------+
|{"total_count": 1...|
+--------------------+



In [67]:
dp=streaming_df.select(from_json(col('value').cast('string'), schema1).alias('test1'))

In [68]:
df_processed = dp.select("test1.*") 

In [69]:
df_processed.writeStream.format("console").start()

24/07/27 19:28:54 WARN ResolveWriteToStream: Temporary checkpoint location created which is deleted normally when the query didn't fail: /tmp/temporary-3678115a-9673-45f7-9472-e6d07d028d2f. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folder is best effort.
24/07/27 19:28:54 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.


-------------------------------------------
Batch: 0
-------------------------------------------
+-----------+--------------------+
|total_count|             results|
+-----------+--------------------+
|       1496|[{"stationcode":"...|
+-----------+--------------------+



In [82]:
df_transformed = df_processed.withColumn("bikes", expr("substring(results, 2, length(results) - 2)"))


In [85]:
df_transformed.writeStream.format("console").start()

24/07/27 19:47:46 WARN ResolveWriteToStream: Temporary checkpoint location created which is deleted normally when the query didn't fail: /tmp/temporary-b4a69f6d-1cb9-47f3-93ac-54400e7aefc3. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folder is best effort.
24/07/27 19:47:46 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.


-------------------------------------------
Batch: 0
-------------------------------------------
+-----------+--------------------+--------------------+--------------------+
|total_count|             results|           timestamp|               bikes|
+-----------+--------------------+--------------------+--------------------+
|       1496|[{"stationcode":"...|2024-07-27 17:39:...|{"stationcode":"4...|
+-----------+--------------------+--------------------+--------------------+



In [35]:
schema = StructType([
    StructField("total_count", StringType(), True),
    StructField("results", StructType([
        StructField("stationcode", IntegerType(), True),
        StructField("name", StringType(), True),
        StructField("is_installed", StringType(), True),
        StructField("capacity", IntegerType(), True),
        StructField("numdocksavailable", IntegerType(), True),
        StructField("numbikesavailable", IntegerType(), True),
        StructField("mechanical", IntegerType(), True),
        StructField("ebike", IntegerType(), True),
        StructField("is_renting", StringType(), True),
        StructField("is_returning", StringType(), True),
        StructField("duedate", DateType(), True),
        StructField("coordonnees_geo", StringType(), True),
        StructField("nom_arrondissement_communes", StringType(), True),
        StructField("code_insee_commune", StringType(), True)]), True)
])

In [75]:
df_parsed = streaming_df.select(from_json(col("value").cast("string"), schema1).alias("bikes"), col("timestamp"))

# Extract the fields from the parsed JSON data
df_processed = df_parsed.select("bikes.*", "timestamp")


In [76]:
df_processed.writeStream.format("console").start()

24/07/27 19:34:22 WARN ResolveWriteToStream: Temporary checkpoint location created which is deleted normally when the query didn't fail: /tmp/temporary-042f77ea-4912-41be-a7f8-ac907ee78302. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folder is best effort.
24/07/27 19:34:22 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.


-------------------------------------------
Batch: 0
-------------------------------------------
+-----------+--------------------+--------------------+
|total_count|             results|           timestamp|
+-----------+--------------------+--------------------+
|       1496|[{"stationcode":"...|2024-07-27 17:39:...|
+-----------+--------------------+--------------------+



In [ ]:
window_duration = '1 minutes'
# Group the tweets by language and timestamp window and count the tweets
df_counts = df_processed.withWatermark("timestamp", window_duration)

#query_df = df_counts.writeStream \
#    .outputMode("complete") \
#    .format("console") \
#    .option("truncate", False) \
#    .start()

query = df_counts.writeStream\
  .trigger(processingTime="1 minutes")\
  .option("checkpointLocation", "checkpoint/")\
  .option("path", ".")\
  .outputMode("append")\
  .start()\
  .awaitTermination()
#df_counts.writeStream.toTable("my_table", checkpointLocation='/Users/rihab.ghrab/Infor-Ideas/Spark')


In [50]:
ds = spark.read.parquet('part-00000-580b3221-ce6c-42b1-8469-4e96b0de5f7a-c000.snappy.parquet')

In [51]:
ds.show()

+--------------------+----------+------------+
|              window|forecastID|record_count|
+--------------------+----------+------------+
|{2024-07-16 10:26...|      NULL|          25|
+--------------------+----------+------------+



In [15]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import from_json, col, explode
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, DoubleType, MapType, ArrayType

# Initialize Spark session
spark = SparkSession.builder \
    .appName("KafkaSparkConsumer") \
    .getOrCreate()

# Kafka parameters
kafka_brokers = "localhost:9092"
kafka_topic = "velib_bikes"

# Define the schema for the JSON data
schema = StructType([
    StructField("total_count", IntegerType(), True),
    StructField("results", ArrayType(StructType([
        StructField("stationcode", StringType(), True),
        StructField("name", StringType(), True),
        StructField("is_installed", StringType(), True),
        StructField("capacity", IntegerType(), True),
        StructField("numdocksavailable", IntegerType(), True),
        StructField("numbikesavailable", IntegerType(), True),
        StructField("mechanical", IntegerType(), True),
        StructField("ebike", IntegerType(), True),
        StructField("is_renting", StringType(), True),
        StructField("is_returning", StringType(), True),
        StructField("duedate", StringType(), True),
        StructField("coordonnees_geo", MapType(StringType(), DoubleType()), True),
        StructField("nom_arrondissement_communes", StringType(), True),
        StructField("code_insee_commune", StringType(), True)
    ])), True)
])

# Read data from Kafka
df_kafka = spark.readStream \
    .format("kafka") \
    .option("kafka.bootstrap.servers", kafka_brokers) \
    .option("subscribe", kafka_topic) \
    .option("failOnDataLoss", "false") \
    .option("startingOffsets", "earliest") \
    .load()

# Extract the value from Kafka message and parse it as JSON
df_parsed = df_kafka.select(from_json(col("value").cast('string'), schema).alias("data"), col('timestamp')) \
    .select("data.*", "timestamp")


In [12]:
df_parsed.printSchema

<bound method DataFrame.printSchema of DataFrame[total_count: int, results: array<struct<stationcode:string,name:string,is_installed:string,capacity:int,numdocksavailable:int,numbikesavailable:int,mechanical:int,ebike:int,is_renting:string,is_returning:string,duedate:string,coordonnees_geo:map<string,double>,nom_arrondissement_communes:string,code_insee_commune:string>>, timestamp: string]>

In [13]:
df_parsed.writeStream.format("console").start()

24/07/28 15:04:36 WARN ResolveWriteToStream: Temporary checkpoint location created which is deleted normally when the query didn't fail: /tmp/temporary-c48bd6b3-cd1d-44db-8cb9-e4ee85958cb0. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folder is best effort.
24/07/28 15:04:36 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.


-------------------------------------------
Batch: 0
-------------------------------------------
+-----------+--------------------+--------------------+
|total_count|             results|           timestamp|
+-----------+--------------------+--------------------+
|       1496|[{9020, Toudouze ...|2024-07-28 15:04:...|
+-----------+--------------------+--------------------+



In [16]:

# Explode the results array into individual rows
df_exploded = df_parsed.select("total_count","timestamp", explode(col("results")).alias("result"))

# Select the necessary fields from the exploded results
df_result = df_exploded.select(
    col("total_count"),
    col("result.stationcode"),
    col("result.name"),
    col("result.is_installed"),
    col("result.capacity"),
    col("result.numdocksavailable"),
    col("result.numbikesavailable"),
    col("result.mechanical"),
    col("result.ebike"),
    col("result.is_renting"),
    col("result.is_returning"),
    col("result.duedate"),
    col("result.coordonnees_geo.lon").alias("longitude"),
    col("result.coordonnees_geo.lat").alias("latitude"),
    col("result.nom_arrondissement_communes"),
    col("result.code_insee_commune"),
    col("timestamp"),
)

# Write the results to the console for debugging (optional)
#query = df_result.writeStream \
#.outputMode("append") \
#    .format("console") \
#    .start()

#query.awaitTermination()


In [23]:
df_result.writeStream.format("console").start()

24/07/28 15:26:25 WARN ResolveWriteToStream: Temporary checkpoint location created which is deleted normally when the query didn't fail: /tmp/temporary-d0a5b8b8-ddb3-4984-b38b-3c4cce4bd508. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folder is best effort.
24/07/28 15:26:25 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.


-------------------------------------------
Batch: 0
-------------------------------------------
+-----------+-----------+------------------+------------+--------+-----------------+-----------------+----------+-----+----------+------------+--------------------+------------------+-----------------+---------------------------+------------------+--------------------+
|total_count|stationcode|              name|is_installed|capacity|numdocksavailable|numbikesavailable|mechanical|ebike|is_renting|is_returning|             duedate|         longitude|         latitude|nom_arrondissement_communes|code_insee_commune|           timestamp|
+-----------+-----------+------------------+------------+--------+-----------------+-----------------+----------+-----+----------+------------+--------------------+------------------+-----------------+---------------------------+------------------+--------------------+
|       1496|       9020|Toudouze - Clauzel|         OUI|      21|               18|         

In [ ]:
window_duration = '1 minutes'
# Group the tweets by language and timestamp window and count the tweets
df_counts = df_result.withWatermark("timestamp", window_duration)

#query_df = df_counts.writeStream \
#    .outputMode("complete") \
#    .format("console") \
#    .option("truncate", False) \
#    .start()

query = df_counts.writeStream\
  .trigger(processingTime="1 minutes")\
  .option("checkpointLocation", "gs://spotify_data_rg_01/checkpoint")\
  .option("path", "gs://spotify_data_rg_01/bikes")\
  .outputMode("append")\
  .start()\
  .awaitTermination()
#df_counts.writeStream.toTable("my_table", checkpointLocation='/Users/rihab.ghrab/Infor-Ideas/Spark')


In [20]:
spark.read.parquet('/home/rihab/Downloads/bikes_part-00000-729a41ea-d2f1-4e83-ac29-8971a3c36247-c000.snappy.parquet').show()

+-----------+-----------+--------------------+------------+--------+-----------------+-----------------+----------+-----+----------+------------+--------------------+------------------+------------------+---------------------------+------------------+--------------------+
|total_count|stationcode|                name|is_installed|capacity|numdocksavailable|numbikesavailable|mechanical|ebike|is_renting|is_returning|             duedate|         longitude|          latitude|nom_arrondissement_communes|code_insee_commune|           timestamp|
+-----------+-----------+--------------------+------------+--------+-----------------+-----------------+----------+-----+----------+------------+--------------------+------------------+------------------+---------------------------+------------------+--------------------+
|       1496|      16107|Benjamin Godard -...|         OUI|      35|               35|                0|         0|    0|       OUI|         OUI|2024-07-28T13:01:...|          2.275

24/07/28 16:06:06 ERROR FileFormatWriter: Aborting job 5cdbe4ea-8bf7-45c0-857b-958e469fd287.
java.io.FileNotFoundException: Unable to find batch gs://spotify_data_rg_01/bikes/_spark_metadata/0
	at org.apache.spark.sql.errors.QueryExecutionErrors$.batchMetadataFileNotFoundError(QueryExecutionErrors.scala:1597)
	at org.apache.spark.sql.execution.streaming.HDFSMetadataLog.applyFnToBatchByStream(HDFSMetadataLog.scala:153)
	at org.apache.spark.sql.execution.streaming.CompactibleFileStreamLog.applyFnInBatch(CompactibleFileStreamLog.scala:207)
	at org.apache.spark.sql.execution.streaming.CompactibleFileStreamLog.foreachInBatch(CompactibleFileStreamLog.scala:193)
	at org.apache.spark.sql.execution.streaming.CompactibleFileStreamLog.$anonfun$compact$3(CompactibleFileStreamLog.scala:235)
	at scala.runtime.java8.JFunction1$mcVJ$sp.apply(JFunction1$mcVJ$sp.java:23)
	at scala.collection.immutable.NumericRange.foreach(NumericRange.scala:75)
	at org.apache.spark.sql.execution.streaming.CompactibleFil